In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


In [2]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [3]:

train_col=df_train.columns
print(train_col)

Index(['id', 'Label', 'ENSG00000242268.2', 'ENSG00000270112.3',
       'ENSG00000167578.15', 'ENSG00000273842.1', 'ENSG00000078237.5',
       'ENSG00000146083.10', 'ENSG00000225275.4', 'ENSG00000158486.12',
       ...
       'ENSG00000238244.3', 'ENSG00000186115.11', 'ENSG00000216352.1',
       'ENSG00000267117.1', 'ENSG00000273233.1', 'ENSG00000105063.17',
       'ENSG00000231119.2', 'ENSG00000280861.1', 'ENSG00000123685.7',
       'ENSG00000181518.3'],
      dtype='object', length=60485)


In [4]:
import math
for column in train_col:
    if (column=='id' or column=='Label'):
        df_train[column]=df_train[column]
    else:
#         df_test[column]=df_test[column].astype(float)
#         df_train[column]=df_train[column].astype(float)
         df_train[column]=np.log2(df_train[column]+1)

In [5]:
test_cols=df_test.columns

In [6]:
   
for column in test_cols:
    if (column=='id'):
        df_test[column]=df_test[column]
    else:
        df_test[column]=np.log2(df_test[column]+1)

In [7]:
var_train=df_train.loc[:,train_col[2:]].var()

In [8]:
temp={}

for i in range(2,len(train_col)):
    temp[train_col[i]]=var_train[i-2]


In [9]:
drop_col_after_variance=[]
for i in temp:
    if temp[i]<0.025:
        drop_col_after_variance.append(i)


In [10]:
df_after_drop_train=df_train.drop(drop_col_after_variance,axis=1, inplace= True)
df_after_drop_test=df_test.drop(drop_col_after_variance,axis=1, inplace= True)

In [11]:
print(len(df_train.columns))
train_newc=df_train.columns
test_newc=df_test.columns
print(test_newc)

23530
Index(['id', 'ENSG00000242268.2', 'ENSG00000167578.15', 'ENSG00000078237.5',
       'ENSG00000146083.10', 'ENSG00000172137.17', 'ENSG00000167700.7',
       'ENSG00000060642.9', 'ENSG00000182141.8', 'ENSG00000269416.4',
       ...
       'ENSG00000253333.1', 'ENSG00000146707.13', 'ENSG00000135094.9',
       'ENSG00000238300.1', 'ENSG00000203729.7', 'ENSG00000009694.12',
       'ENSG00000186115.11', 'ENSG00000105063.17', 'ENSG00000231119.2',
       'ENSG00000123685.7'],
      dtype='object', length=23529)


In [12]:
from sklearn import preprocessing 
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1)) 
# scaled_df_train = min_max_scaler.fit_transform(df_train)
scaled_df_test = min_max_scaler.fit_transform(df_test)



In [13]:
print ("\nAfter min max Scaling : \n", scaled_df_test[0])


After min max Scaling : 
 [0.         0.         0.52284226 ... 0.61244195 0.16442198 0.30478565]


In [16]:
print ("\nAfter min max Scaling : \n", scaled_df_test)
x = df_train[train_newc[2:]].values
scaled_df_train = min_max_scaler.fit_transform(x)
print ("\nAfter min max Scaling : \n", scaled_df_train)


After min max Scaling : 
 [[0.         0.         0.52284226 ... 0.61244195 0.16442198 0.30478565]
 [0.01428571 0.19382908 0.62290016 ... 0.73156455 0.30651927 0.322804  ]
 [0.02857143 0.02334698 0.57941813 ... 0.74851551 0.18014273 0.06927184]
 ...
 [0.97142857 0.         0.29351973 ... 0.68912147 0.16219335 0.27639978]
 [0.98571429 0.         0.99932856 ... 0.83009732 0.46396619 0.45401662]
 [1.         0.03682752 0.22952675 ... 0.53707463 0.16030565 0.19930665]]

After min max Scaling : 
 [[0.67860919 0.69449131 0.5895407  ... 0.8062757  0.01322989 0.84018786]
 [0.         0.30315464 0.27674461 ... 0.83188483 0.23765982 0.12622318]
 [0.         0.20876788 0.57524338 ... 0.60932618 0.1766162  0.26308282]
 ...
 [0.         0.22470112 0.59355616 ... 0.62110777 0.10187    0.06759435]
 [0.         0.4337966  0.46759727 ... 0.53130371 0.16232231 0.18765424]
 [0.         0.19346217 0.53880659 ... 0.58293568 0.41473421 0.15425069]]


In [19]:
X = df_train.iloc[:,2:23530]  #independent columns
y = df_train.iloc[:,1]    #target column i.e price range
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=100)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(100,'Score')) 

                    Specs      Score
9101   ENSG00000107159.11  26.247872
14738  ENSG00000181577.14  18.505226
8121   ENSG00000170381.11  18.032017
10990   ENSG00000133636.9  17.544432
15937  ENSG00000158516.10  17.317291
...                   ...        ...
167    ENSG00000105048.15   9.115793
20813  ENSG00000006611.14   9.098489
4752    ENSG00000163082.9   9.068913
1505    ENSG00000269353.1   8.982950
22354  ENSG00000149452.14   8.955489

[100 rows x 2 columns]


In [20]:
imp=featureScores.nlargest(100,'Score')
imp_feat=imp["Specs"]
imp_cols=[]
for i in range(len(imp_feat)):
    imp_cols.append(imp_feat.iloc[i])


In [21]:
cols_drop1= list(set(train_newc).difference(set(imp_cols)))

In [22]:
# print(cols_drop1)
cols_drop_final=[]
for i in cols_drop1:
    if i =="id" or i=="Label":
        print(i)
    else:
        cols_drop_final.append(i)

Label
id


In [24]:
df_after_drop_train1=df_train.drop(cols_drop_final,axis=1, inplace= True)
df_after_drop_test1=df_test.drop(cols_drop_final,axis=1, inplace= True)
        

In [25]:
ftrain_col=df_train.columns
print(ftrain_col)

Index(['id', 'Label', 'ENSG00000105048.15', 'ENSG00000149124.9',
       'ENSG00000269353.1', 'ENSG00000253417.4', 'ENSG00000069011.14',
       'ENSG00000228952.1', 'ENSG00000239093.1', 'ENSG00000072041.15',
       ...
       'ENSG00000196188.9', 'ENSG00000184349.11', 'ENSG00000214748.2',
       'ENSG00000185519.8', 'ENSG00000149452.14', 'ENSG00000145826.7',
       'ENSG00000162078.10', 'ENSG00000100604.11', 'ENSG00000251985.1',
       'ENSG00000224525.2'],
      dtype='object', length=102)


In [26]:
independent_attri=[]
for i in range(2,len(ftrain_col)):
    independent_attri.append(ftrain_col[i])
print(independent_attri)

['ENSG00000105048.15', 'ENSG00000149124.9', 'ENSG00000269353.1', 'ENSG00000253417.4', 'ENSG00000069011.14', 'ENSG00000228952.1', 'ENSG00000239093.1', 'ENSG00000072041.15', 'ENSG00000114529.11', 'ENSG00000016402.11', 'ENSG00000226141.1', 'ENSG00000088836.11', 'ENSG00000159184.7', 'ENSG00000234311.1', 'ENSG00000088002.10', 'ENSG00000260196.1', 'ENSG00000089356.15', 'ENSG00000122711.7', 'ENSG00000177679.15', 'ENSG00000102109.8', 'ENSG00000136943.9', 'ENSG00000144366.14', 'ENSG00000163082.9', 'ENSG00000117983.16', 'ENSG00000167771.5', 'ENSG00000165023.5', 'ENSG00000185686.16', 'ENSG00000157765.10', 'ENSG00000204060.5', 'ENSG00000007306.13', 'ENSG00000170381.11', 'ENSG00000006659.11', 'ENSG00000140274.12', 'ENSG00000074211.12', 'ENSG00000210741.1', 'ENSG00000107159.11', 'ENSG00000106541.10', 'ENSG00000169297.7', 'ENSG00000154646.7', 'ENSG00000109758.7', 'ENSG00000160182.2', 'ENSG00000256310.1', 'ENSG00000140107.10', 'ENSG00000133636.9', 'ENSG00000206878.1', 'ENSG00000267795.4', 'ENSG0000014

In [33]:
ftest_col=df_test.columns

independent_attri_test=[]
for i in range(1,len(ftest_col)):
    independent_attri_test.append(ftest_col[i])
print(independent_attri_test)

['ENSG00000105048.15', 'ENSG00000149124.9', 'ENSG00000269353.1', 'ENSG00000253417.4', 'ENSG00000069011.14', 'ENSG00000228952.1', 'ENSG00000239093.1', 'ENSG00000072041.15', 'ENSG00000114529.11', 'ENSG00000016402.11', 'ENSG00000226141.1', 'ENSG00000088836.11', 'ENSG00000159184.7', 'ENSG00000234311.1', 'ENSG00000088002.10', 'ENSG00000260196.1', 'ENSG00000089356.15', 'ENSG00000122711.7', 'ENSG00000177679.15', 'ENSG00000102109.8', 'ENSG00000136943.9', 'ENSG00000144366.14', 'ENSG00000163082.9', 'ENSG00000117983.16', 'ENSG00000167771.5', 'ENSG00000165023.5', 'ENSG00000185686.16', 'ENSG00000157765.10', 'ENSG00000204060.5', 'ENSG00000007306.13', 'ENSG00000170381.11', 'ENSG00000006659.11', 'ENSG00000140274.12', 'ENSG00000074211.12', 'ENSG00000210741.1', 'ENSG00000107159.11', 'ENSG00000106541.10', 'ENSG00000169297.7', 'ENSG00000154646.7', 'ENSG00000109758.7', 'ENSG00000160182.2', 'ENSG00000256310.1', 'ENSG00000140107.10', 'ENSG00000133636.9', 'ENSG00000206878.1', 'ENSG00000267795.4', 'ENSG0000014

In [34]:
test_data=df_test[ftest_col]


In [35]:
#applying PCA on training data
scalar = StandardScaler() 
x=df_train[independent_attri]
# fitting 
scalar.fit(x) 
scaled_df = scalar.transform(x) 

from sklearn.decomposition import PCA
p = PCA(n_components = 2) 
p.fit(scaled_df) 
x_axis = p.transform(scaled_df) 
x_std=p.fit_transform(scaled_df)
x_std.shape
print(len(x_std))

279


In [36]:
#applying PCA on test data
scalar1 = StandardScaler() 
x_test=df_test[independent_attri_test]

# fitting 
scalar1.fit(x_test) 
scaled_df1 = scalar1.transform(x_test) 

from sklearn.decomposition import PCA
p1 = PCA(n_components = 2) 
p1.fit(scaled_df1) 
x_axis1 = p1.transform(scaled_df1) 
x_std1=p1.fit_transform(scaled_df1)
x_std1.shape
print(len(x_std1))

71


In [84]:
y1=df_train.Label.values
x1=x_std
# x1=df_train[independent_attri]
xtrain, xtest, ytrain, ytest = train_test_split(x1,y1,test_size=0.2)

In [88]:
from sklearn.tree import DecisionTreeClassifier
descision_clf=DecisionTreeClassifier(criterion = "entropy", random_state = 1, max_depth = 5)
descision_clf=descision_clf.fit(xtrain,ytrain)
y_pred_dt = descision_clf.predict(x_std1)
print("Lables after prediction are: ", y_pred_dt)

Lables after prediction are:  [ 1 -1  1  1 -1  1 -1 -1  1  1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1  1  1  1
 -1 -1  1  1 -1  1  1  1  1 -1  1  1 -1  1  1  1  1 -1  1 -1 -1  1  1 -1
  1  1  1  1 -1 -1  1  1 -1 -1  1 -1 -1  1  1 -1  1 -1  1 -1 -1  1  1]


In [86]:
list1=df_test["id"]
output_list=[]
test_c=["id","Label"]
output_list.append(test_c)
for i in range(len(list1)):
    t=[]
    t.append(list1[i])
    t.append(y_pred_mlp[i])
    output_list.append(t)
    

In [87]:
print(output_list)
import csv
with open('output_dt1.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(output_list)
  

[['id', 'Label'], [3001, 1], [3002, 1], [3003, 1], [3004, 1], [3005, -1], [3006, -1], [3007, -1], [3008, 1], [3009, 1], [3010, 1], [3011, 1], [3012, 1], [3013, -1], [3014, -1], [3015, 1], [3016, 1], [3017, -1], [3018, 1], [3019, 1], [3020, 1], [3021, 1], [3022, 1], [3023, 1], [3024, 1], [3025, 1], [3026, -1], [3027, 1], [3028, -1], [3029, 1], [3030, 1], [3031, 1], [3032, 1], [3033, 1], [3034, 1], [3035, 1], [3036, 1], [3037, 1], [3038, 1], [3039, 1], [3040, 1], [3041, 1], [3042, -1], [3043, 1], [3044, -1], [3045, 1], [3046, 1], [3047, 1], [3048, 1], [3049, -1], [3050, 1], [3051, 1], [3052, 1], [3053, -1], [3054, -1], [3055, 1], [3056, 1], [3057, 1], [3058, 1], [3059, -1], [3060, 1], [3061, -1], [3062, 1], [3063, 1], [3064, 1], [3065, 1], [3066, -1], [3067, 1], [3068, 1], [3069, 1], [3070, -1], [3071, 1]]
